In [1]:
# add parent directory to path: enable import from parent dir
import sys
sys.path.append('../')

from environment import SmartBrokerEnv
from agents.a2c import A2C
from networks.a2c.actor import Actor
from networks.a2c.critic import Critic

import matplotlib.pyplot as plt
import torch

norm_cols = ['Volume XRP']
cols = ['date'] + ['open', 'high', 'low', 'close', f'Volume XRP']
batch_dur = 30
env = SmartBrokerEnv(
    batch_dur=batch_dur,
    df_info={
        'start_date': '2021-01-01',
        'end_date': '2021-02-01',
        'norm_cols': norm_cols,
        'cols': cols,
    },
    portfolio={

    },
)

actor_model = Actor(
    state_dim=batch_dur*4+3,
    action_dim=3,
)
critic_model = Critic(state_dim=batch_dur*4+3)
a2c = A2C(
    env=env, 
    actor=actor_model,
    critic=critic_model,
    n_actns=3,
    actor_optmz=torch.optim.Adam(actor_model.parameters(), lr=0.0008),
    critic_optmz=torch.optim.Adam(critic_model.parameters(), lr=0.0005),
    hyprprms={
        'gamma': 0.9995,
    },
    log_freq=1,
    mdl_pth='../models/a2c_nn_3',
    load_models=False,
)

In [ ]:
a2c.run(1000)


Ep: 0 | TS: 18108 | L: 0.77 | R: -18.09 | P: 0.68 | R.Avg P: 0.68 | NW: 100.68 | R.Avg NW: 100.68 | R.U: 162
Ep: 1 | TS: 28458 | L: 1.562 | R: -14.59 | P: 28.33 | R.Avg P: 14.5 | NW: 128.33 | R.Avg NW: 114.51 | R.U: 176
Ep: 2 | TS: 49885 | L: 0.871 | R: -35.48 | P: -0.44 | R.Avg P: 9.52 | NW: 99.56 | R.Avg NW: 109.52 | R.U: 173
Ep: 3 | TS: 62625 | L: 0.709 | R: -25.81 | P: 31.95 | R.Avg P: 15.13 | NW: 131.95 | R.Avg NW: 115.13 | R.U: 178
Ep: 4 | TS: 70976 | L: 1.222 | R: -45.85 | P: -0.33 | R.Avg P: 12.04 | NW: 99.67 | R.Avg NW: 112.04 | R.U: 169
Ep: 5 | TS: 110095 | L: 0.954 | R: -21.59 | P: 42.01 | R.Avg P: 17.03 | NW: 142.01 | R.Avg NW: 117.03 | R.U: 181
Ep: 6 | TS: 131145 | L: 0.895 | R: -40.2 | P: 7.47 | R.Avg P: 15.67 | NW: 107.47 | R.Avg NW: 115.67 | R.U: 182
Ep: 7 | TS: 162954 | L: 1.008 | R: -55.55 | P: -5.37 | R.Avg P: 13.04 | NW: 94.63 | R.Avg NW: 113.04 | R.U: 177
Ep: 8 | TS: 179475 | L: 1.033 | R: -31.04 | P: 27.16 | R.Avg P: 14.61 | NW: 127.16 | R.Avg NW: 114.61 | R.U: 1

In [ ]:
rewards, profits, actions = a2c.evaluate(start_dt='2021-01-30 15:15', duration=3000, show_pred=False)

In [ ]:
rewards, profits, actions = a2c.evaluate(start_dt='2021-01-08 05:15', duration=3000, show_pred=False)

## Visualizations

### Training 

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(16, 12))
r_avg_rewards = []
r_avg_profits = []
r_avg_bal = []
r_avg_units_held = []
r_avg_loss = []
r_avg_net_worth = []
count = 0

for _, log in a2c.logs.items():
    r_avg_rewards.append(log['r_avg_reward'])
    r_avg_profits.append(log['r_avg_profit'])
    r_avg_bal.append(log['r_avg_bal'])
    r_avg_units_held.append(log['r_avg_units_held'])
    r_avg_loss.append(log['r_avg_loss'])
    r_avg_net_worth.append(log['r_avg_net_worth'])
    count += 1

ax[0][0].plot(range(count), r_avg_loss)
ax[0][0].set_title('Rolling avg loss per episode')

ax[0][1].plot(range(count), r_avg_rewards)
ax[0][1].set_title('Rolling avg reward per episode')

ax[1][0].plot(range(count), r_avg_profits)
ax[1][0].set_title('Rolling avg profit per episode')

ax[1][1].plot(range(count), r_avg_units_held)
ax[1][1].set_title('Rolling avg units held per episode')

ax[2][0].plot(range(count), r_avg_net_worth)
ax[2][0].set_title('Rolling avg net worth per episode')

ax[2][1].plot(range(count), r_avg_bal)
ax[2][1].set_title('Rolling avg balance per episode')

In [ ]:
# torch.save(a2c.actor.state_dict(), '../models/a2c_nn_3/actor')
# torch.save(a2c.critic.state_dict(), '../models/a2c_nn_3/critic')

In [ ]:
# import pickle
# with open("../pickles/a2c_logs_main.pickle","wb") as f:
#     temp = dict(a2c.logs)
#     pickle.dump(temp, f, pickle.HIGHEST_PROTOCOL)